In [2]:
import utils
from tqdm.notebook import tqdm
import sample as sampler
import numpy as np
dimension = 512
k = 16  # alpha error correcting parameter

## Utils

In [3]:
import numpy as np
def subset_n_a_k(n, a, k):
    result = 0
    for i in range(k):
        result += np.log2(n-i) - np.log2(n-a-i)
    return result

In [4]:
def generate_puzzle_local_search(new_bs, new_isometric_matrixes, k=160):
    # generate matrix with row vectors from new_isometric_matrixes that orthognal to orignal template
    # k indicates the number of row vectors to be selected from each isometric matrix
    sub_matrixes = []
    for b, isometric_matrix in zip(new_bs, new_isometric_matrixes):
        indexb = np.where(np.abs(b) < 0.1)[0]
        # get random subset of shape n of indexb
        indexbb = np.random.choice(indexb, k, replace=False)
        sub_matrixes.append(isometric_matrix[indexbb, :])

    return np.concatenate(sub_matrixes, axis=0)

def generate_puzzle_local_search_random(new_bs, new_isometric_matrixes, k = 160):
    # generate matrix with row vector from new_isometric_matrixes randomly
    # k indicates the number of row vectors to be selected from each isometric matrix
    sub_matrixes = []
    for b, isometric_matrix in zip(new_bs, new_isometric_matrixes):
        # get random subset of shape n of indexb
        indexbb = np.random.choice(b.shape[0], k, replace=False)
        sub_matrixes.append(isometric_matrix[indexbb, :])

    return np.concatenate(sub_matrixes, axis=0)

In [5]:
import time
def local_search_each_runtime(dimension, k, sub_matrix, disable_flag=False, each_runtime=1000):
    # test for running time
    times = 0
    # time start
    start = time.time()
    for _ in (pbar:=tqdm(range(each_runtime), disable=disable_flag, leave=True)):
        theta = sampler.random_vector_with_k_nonzero_elements(dimension, k)
        flag = False
        for _ in range(800):
            tmpb = sub_matrix @ theta
            # get the norm of tmpb
            normb = np.linalg.norm(tmpb)
            # get the index of tmpb where are not zero and iterate on it
            nonzero = np.where(np.abs(theta) > 0.1)[0]
            thetab = None
            tmp_sub_matrix = sub_matrix.copy()
            # set columns in non_zero positions with 10 * np.ones
            tmp_sub_matrix[:, nonzero] = 10 * np.ones((sub_matrix.shape[0], len(nonzero)))
            for j1 in nonzero:
                tmpbb = tmpb - theta[j1] * sub_matrix[:, j1]
                # add
                tmpbbb = tmp_sub_matrix + tmpbb[:, np.newaxis]
                normbbb = np.linalg.norm(tmpbbb, axis=0)
                # get min value and index of normbbb
                min_index = np.argmin(normbbb)
                min_value = normbbb[min_index]
                if min_value < normb:
                    thetab = theta.copy()
                    thetab[j1] = 0
                    thetab[min_index] = 1
                    normb = min_value
                # sub
                tmpbbb =  tmpbb[:, np.newaxis] - tmp_sub_matrix
                normbbb = np.linalg.norm(tmpbbb, axis=0)
                # get min value and index of normbbb
                min_index = np.argmin(normbbb)
                min_value = normbbb[min_index]
                if min_value < normb:
                    thetab = theta.copy()
                    thetab[j1] = 0
                    thetab[min_index] = -1
                    normb = min_value
            # pbar update
            pbar.set_postfix({"norm": normb})
            if thetab is not None:
                theta = thetab
                continue
            else:
                flag = True
                break
        if flag:
            times += 1
        pbar.set_postfix({"Reach ending times": times})
    end = time.time()
    slap_time = end - start
    return slap_time

In [6]:
def local_search_one_time(dimension, k, sub_matrix, each_run_time = 800, disable_flag=False, **kwargs):
    # outer itration = 1, output whether the solution is parallel to original codeword(vector a)
    a = kwargs.get("a", None)
    # threshold = kwargs.get("threshold", 1e-1) / np.sqrt(dimension) * np.sqrt(k)
    theta = sampler.random_vector_with_k_nonzero_elements(dimension, k)
    thetab = None
    normb = 16
    for j in range(each_run_time):
        tmpb = sub_matrix @ theta
        # get the norm of tmpb
        normb = np.linalg.norm(tmpb)
        # get the index of tmpb where are not zero and iterate on it
        nonzero = np.where(np.abs(theta) > 0.1)[0]
        thetab = None
        tmp_sub_matrix = sub_matrix.copy()
        # set columns in non_zero positions with 10 * np.ones
        tmp_sub_matrix[:, nonzero] = 10 * np.ones((sub_matrix.shape[0], len(nonzero)))
        for j1 in nonzero:
            tmpbb = tmpb - theta[j1] * sub_matrix[:, j1]
            # add
            tmpbbb = tmp_sub_matrix + tmpbb[:, np.newaxis]
            normbbb = np.linalg.norm(tmpbbb, axis=0)
            # get min value and index of normbbb
            min_index = np.argmin(normbbb)
            min_value = normbbb[min_index]
            if min_value < normb:
                thetab = theta.copy()
                thetab[j1] = 0
                thetab[min_index] = 1
                normb = min_value
            # sub
            tmpbbb =  tmpbb[:, np.newaxis] - tmp_sub_matrix
            normbbb = np.linalg.norm(tmpbbb, axis=0)
            # get min value and index of normbbb
            min_index = np.argmin(normbbb)
            min_value = normbbb[min_index]
            if min_value < normb:
                thetab = theta.copy()
                thetab[j1] = 0
                thetab[min_index] = -1
                normb = min_value
        if thetab is not None:
            theta = thetab
            continue
        else:
            break
    if np.allclose(theta - a, 0, 1e-1) or np.allclose(theta + a, 0, 1e-1):
        return True
    else:
        return False

## TEST

### Test for getting 3 sketches

average running time

In [36]:
# time testing
whole_time = 0
tmp_inner = 100
tmp_outer = 10
tmpk = 190
error_rate = 0.5
for _ in (pbar:= tqdm(range(tmp_outer), desc="Outer loop")):
    # need to control n to indicate the number of sketches(default = 3)
    c, bs, isometric_matrixes, coserrors = sampler.generate_puzzle_n(dimension, k, error_rate=error_rate, n=3, disable_tqdm=True)
    new_isometric_matrixes = [isometric_matrixes[i] @ isometric_matrixes[0].T for i in range(1, len(isometric_matrixes))]
    new_bs = [bs[i] for i in range(1, len(bs))]
    # need to control k (set tmpk) to indicate the number of row vectors to be selected from each isometric matrix
    whole_time += local_search_each_runtime(dimension, k, generate_puzzle_local_search_random(new_bs, new_isometric_matrixes, k=tmpk), disable_flag=False, each_runtime=tmp_inner)
    pbar.set_postfix({"Whole Time": whole_time})
print("Average time for each inner iteration:", whole_time / (tmp_outer * tmp_inner))
print("Whole: Time", whole_time)

Outer loop:   0%|          | 0/10 [00:00<?, ?it/s]

  0%|          | 0/100 [00:00<?, ?it/s]

  0%|          | 0/100 [00:00<?, ?it/s]

  0%|          | 0/100 [00:00<?, ?it/s]

  0%|          | 0/100 [00:00<?, ?it/s]

...

  0%|          | 0/100 [00:00<?, ?it/s]

  0%|          | 0/100 [00:00<?, ?it/s]

  0%|          | 0/100 [00:00<?, ?it/s]

  0%|          | 0/100 [00:00<?, ?it/s]

Average time for each inner iteration: 0.24155511951446534
Whole: Time 241.55511951446533


average success rate

In [30]:
# 0.035 -> 2.8 degrees; 0.07-> 5.6 degrees; 0.108 -> 8.6 degrees; 0.13 -> 11 degrees;  
# 0.18 -> 14 degrees; 0.25 -> 19 degrees; 0.35 -> 26 degrees; 0.4 -> 30 degrees; 0.5 -> 36 degrees; 
error_rate_iter_arrays = [
    [0.0, [100, 110, 120, 130, 140, 160, 180, 200]],
    [0.108, [120, 130, 140, 150, 160]],
    [0.18, [130, 140, 150, 160]],
    [0.25, [130, 140, 150, 160]],
    [0.35, [160, 170, 180, 190]],
    [0.4, [160, 170, 180, 190]],
    [0.5, [180, 190, 200, 220, 240]],
]
# error_rate = 0.5
# iter_array = [180, 190, 200, 220, 240, 260]
iteration_number = 20000
for error_rate_iter_array in error_rate_iter_arrays:
    error_rate = error_rate_iter_array[0]
    iter_array = error_rate_iter_array[1]
    iter_array.reverse()
    for tmpk in iter_array:        
        success_num = 0
        run_time = iteration_number
        # for efficiency, we generate n=4*dimension + 1 sketches from the same original template with noise, and 
        # test at each time randomly selecting three sketches from this n sketches, using this three sketches as our puzzle
        c, bs, isometric_matrixes, coserrors = sampler.generate_puzzle_n(dimension, k, error_rate=error_rate, n= 4 * dimension + 1, disable_tqdm=False)
        for i in (pbar:=tqdm(range(iteration_number))):
            # random select three templates from new_isometric_matrixes and get new_bs and new_isometric_matrixes
            index = np.random.choice(len(bs), 3, replace=False)
            new_bs = [bs[index[1]], bs[index[2]]]
            new_isometric_matrixes = [isometric_matrixes[index[i+1]] @ isometric_matrixes[index[0]].T for i in range(len(new_bs))]
            tmp_result = local_search_one_time(dimension, k, generate_puzzle_local_search(new_bs, new_isometric_matrixes, k=tmpk), iteration_number=iteration_number, disable_flag=True, a=bs[index[0]], threshold=error_rate * 1.3)
            if tmp_result:
                success_num += 1
            pbar.set_postfix({"Success Rate": success_num/(i+1), "Success Number": success_num})
            # break if getting enough success times(for efficiency)
            if success_num > 100 and i > 1000:
                run_time = i + 1
                break
        # store threshold_array in file with name "threshold_array_k" in directory "data"
        np.save(f"data_with_noise_3/threshold_array_{tmpk}_{error_rate}_success_num_run_time", [success_num, run_time])

Generating Puzzles:   0%|          | 0/2049 [00:00<?, ?it/s]

  0%|          | 0/20000 [00:00<?, ?it/s]

Generating Puzzles:   0%|          | 0/2049 [00:00<?, ?it/s]

  0%|          | 0/20000 [00:00<?, ?it/s]

Generating Puzzles:   0%|          | 0/2049 [00:00<?, ?it/s]

...

  0%|          | 0/20000 [00:00<?, ?it/s]

Generating Puzzles:   0%|          | 0/2049 [00:00<?, ?it/s]

  0%|          | 0/20000 [00:00<?, ?it/s]

Generating Puzzles:   0%|          | 0/2049 [00:00<?, ?it/s]

KeyboardInterrupt: 

### Estimate
how many iteration times need to output original template, multiply inner iteration time of local search algorithm is the estimated time for running whole algorithm 

In [27]:
import numpy as np
import os
import math

# 0.035 -> 2.8 degrees; 0.07-> 5.6 degrees; 0.108 -> 8.6 degrees; 0.13 -> 11 degrees;  
# 0.18 -> 14 degrees; 0.25 -> 19 degrees; 0.35 -> 26 degrees; 0.4 -> 30 degrees; 0.5 -> 36 degrees; 
error_rate = 0.4
for i in [90, 100, 110, 120, 130, 140, 150, 160, 170, 180, 190, 200, 210, 220, 230, 240, 250, 260]:
    threshold_array = []
    if os.path.exists("data_with_noise_3_history/threshold_array_{}_{}_success_num_run_time.npy".format(i, error_rate)):
        results = np.load("data_with_noise_3_history/threshold_array_{}_{}_success_num_run_time.npy".format(i, error_rate))
        success_num = results[0]
        run_time = results[1]
        whole_run_num = run_time
        if success_num == 0: continue
        print("{}".format(i), "success rate:{:.6f}".format(success_num / whole_run_num), " success number:", success_num, " inverse of success rate: ", whole_run_num / success_num, " Expected iteration number: ", math.ceil(whole_run_num / success_num * 2 ** (subset_n_a_k(512, 16, i) * 2)))
        continue

160 success rate:0.000100  success number: 1  inverse of success rate:  10000.0  Expected iteration number:  2005240227
170 success rate:0.001500  success number: 30  inverse of success rate:  666.6666666666666  Expected iteration number:  343186353
180 success rate:0.002500  success number: 50  inverse of success rate:  400.0  Expected iteration number:  543962987
190 success rate:0.004650  success number: 93  inverse of success rate:  215.05376344086022  Expected iteration number:  796446624
200 success rate:0.005408  success number: 101  inverse of success rate:  184.9009900990099  Expected iteration number:  1926264224


###  Test for getting (tmpk + 1) sketches

run time

In [1]:
whole_time = 0
tmp_inner = 100
tmp_outer = 10
tmpk = 160
error_rate = 0.108
for _ in (pbar:= tqdm(range(tmp_outer), desc="Outer loop")):
    c, bs, isometric_matrixes, coserrors = sampler.generate_puzzle_n(dimension, k, error_rate=error_rate, n=tmpk + 1, disable_tqdm=True)
    new_isometric_matrixes = [isometric_matrixes[i] @ isometric_matrixes[0].T for i in range(1, len(isometric_matrixes))]
    new_bs = [bs[i] for i in range(1, len(bs))]
    puzzle = generate_puzzle_local_search_random(new_bs, new_isometric_matrixes, k=1)
    whole_time += local_search_each_runtime(dimension, k, puzzle, disable_flag=True, each_runtime=tmp_inner)
    pbar.set_postfix({"Whole Time": whole_time})
print("Average inner running time: ", whole_time / (tmp_outer * tmp_inner))
print("Whole Time: ", whole_time)

NameError: name 'tqdm' is not defined

average success rate

In [45]:
# 0.035 -> 2.8 degrees; 0.07-> 5.6 degrees; 0.108 -> 8.6 degrees; 0.13 -> 11 degrees;  
# 0.18 -> 14 degrees; 0.25 -> 19 degrees; 0.35 -> 26 degrees; 0.4 -> 30 degrees; 0.5 -> 36 degrees; 

# 0.035 -> 2.8 degrees; 0.07-> 5.6 degrees; 0.108 -> 8.6 degrees; 0.13 -> 11 degrees;  
# 0.18 -> 14 degrees; 0.25 -> 19 degrees; 0.35 -> 26 degrees; 0.4 -> 30 degrees; 0.5 -> 36 degrees; 
error_rate_iter_arrays = [
    [0.0, [130, 140, 150, 160, 170, 180]],
    [0.108, [150, 160, 170]],
    [0.18, [150, 160, 170]],
    [0.25, [150, 160, 170]],
    [0.35, [180, 190, 200, 210, 220]],
    [0.4, [180, 190, 200, 210, 220]],
    [0.5, [250, 260, 270, 280, 290, 300]],
]
iteration_number = 20000
for error_rate_iter_array in error_rate_iter_arrays:
    error_rate = error_rate_iter_array[0]
    iter_array = error_rate_iter_array[1]        # true tmpk = 2 * iter_array
    iter_array.reverse()
    for tmpk in iter_array:        
        tmpk *= 2
        success_num = 0
        run_time = iteration_number
        t_thrshold = 0
        # for efficiency, we generate 4 * dimension + 1 sketches from the same original template with noise, take first sketch and
        # test at each time randomly selecting tmpk sketches from these sketches, using these sketches as our puzzle
        c, bs, isometric_matrixes, coserrors = sampler.generate_puzzle_n(dimension, k, error_rate=error_rate, n = 4 * dimension + 1, disable_tqdm=False)
        new_isometric_matrixes = [isometric_matrixes[i] @ isometric_matrixes[0].T for i in range(1, len(isometric_matrixes))]
        new_bs = [bs[i] for i in range(1, len(bs))]
        a = bs[0]
        for i in (pbar:=tqdm(range(iteration_number))):
            index = np.random.choice(len(new_bs), tmpk, replace=False)
            tmp_new_isometric_matrixes = [new_isometric_matrixes[index[i]] for i in range(0, tmpk)]
            tmp_new_bs = [new_bs[index[i]] for i in range(0, tmpk)]
            tmp_result = local_search_one_time(dimension, k, generate_puzzle_local_search(tmp_new_bs, tmp_new_isometric_matrixes, k=1), iteration_number=iteration_number, disable_flag=True, a=bs[0], threshold=error_rate * 1.3)
            if tmp_result:
                success_num += 1
            pbar.set_postfix({"Success Rate": success_num/(i+1), "Success Number": success_num})
            if success_num > 100 and i > 1000:
                run_time = i
                break
        np.save(f"data_with_noise_k/threshold_array_{tmpk}_{error_rate}_success_num_run_time", [success_num, run_time])

Generating Puzzles:   0%|          | 0/2049 [00:00<?, ?it/s]

  0%|          | 0/20000 [00:00<?, ?it/s]

KeyboardInterrupt: 

### Estimate
how many iteration times need to output original template, multiply inner iteration time of local search algorithm is the estimated time for running whole algorithm 

In [46]:
import numpy as np
import os
import math

# 0.035 -> 2.8 degrees; 0.07-> 5.6 degrees; 0.108 -> 8.6 degrees; 0.13 -> 11 degrees;  
# 0.18 -> 14 degrees; 0.25 -> 19 degrees; 0.35 -> 26 degrees; 0.4 -> 30 degrees; 0.5 -> 36 degrees; 

error_rate = 0.0
for i in [90, 100, 110, 120, 130, 140, 150, 160, 170, 180, 190, 200, 210, 220, 230, 240, 250, 260, 270, 280, 290, 300]:
    i = i * 2
    if os.path.exists("data_with_noise_k/threshold_array_{}_{}_success_num_run_time.npy".format(i, error_rate)):
        results = np.load("data_with_noise_k/threshold_array_{}_{}_success_num_run_time.npy".format(i, error_rate))
        success_num = results[0]
        run_time = results[1]
        whole_run_num = run_time
        if success_num == 0: continue
        print("{}".format(i), "success rate:{:.6f}".format(success_num / whole_run_num), " success number:", success_num, " inverse of success rate: ", whole_run_num / success_num, " Expected iteration number: ", math.ceil(whole_run_num / success_num * 2 ** (subset_n_a_k(512, 16, 1) * i)))
        continue

260 success rate:0.000600  success number: 12  inverse of success rate:  1666.6666666666667  Expected iteration number:  6409184
280 success rate:0.001350  success number: 27  inverse of success rate:  740.7407407407408  Expected iteration number:  5375092
300 success rate:0.002200  success number: 44  inverse of success rate:  454.54545454545456  Expected iteration number:  6223901
320 success rate:0.002900  success number: 58  inverse of success rate:  344.82758620689657  Expected iteration number:  8909494
340 success rate:0.006110  success number: 101  inverse of success rate:  163.65346534653466  Expected iteration number:  7978880
360 success rate:0.010451  success number: 101  inverse of success rate:  95.68316831683168  Expected iteration number:  8802742
